In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer


from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_white, het_breuschpagan, het_goldfeldquandt, linear_reset
import geopandas as gpd
from fiona.drvsupport import supported_drivers

In [2]:
supported_drivers['KML'] = 'rw'

dubai_area_coordinates = gpd.read_file('Community.kml', driver='KML')

In [3]:
df = pd.read_csv('transactions-2023-01-11.csv')

In [4]:
def drop_excess_columns(data):
    # Drop high cardinality columns
    data = data.drop(columns=["Transaction Number", "Property ID", "Transaction Size (sq.m)", "Parking", "Project"])
    # Drop low-cardinality columns
    data = data.drop(columns=["Registration type", "Is Free Hold?", "Master Project"])
    # Drop leaky columns
    data = data.drop(columns=["Transaction sub type", "Property Type", "Room(s)", "No. of Buyer", "No. of Seller"])
    return data


In [5]:
df = drop_excess_columns(df)

In [6]:
def get_oil_price():
    # data from https://tradingeconomics.com/commodity/crude-oil
    r = requests.get('https://markets.tradingeconomics.com/chart?s=cl1:com&interval=1d&span=5y&securify=new&url=/commodity/crude-oil&AUTH=6ojuXIi3c32ADoduSeFTjHLA0xnR6eXyR89j7z4RAvaU2wRLFQjriB9SIi7TZnd%2F&ohlc=0')
    
    oil_data = pd.DataFrame(r.json()['series'][0]['data']).rename(columns={"y": "Oil Price"})
    oil_data = oil_data[(oil_data['date'] > '2021-03-01') & (oil_data['date'] < '2022-02-04')]

    return oil_data[['date', 'Oil Price']]
    

In [7]:
oil_data = get_oil_price()
oil_data

,date,Oil Price
853,2021-03-01T00:00:00,60.64
854,2021-03-02T00:00:00,59.75
855,2021-03-03T00:00:00,61.28
856,2021-03-04T00:00:00,63.83
857,2021-03-05T00:00:00,66.09
...,...,...
1089,2022-01-28T00:00:00,85.43
1090,2022-01-31T00:00:00,86.49
1091,2022-02-01T00:00:00,86.46
1092,2022-02-02T00:00:00,86.69


In [8]:
def drop_period_after_war(data):
    war_date = '2022-02-24'
    return data[data['Transaction Date'] < war_date]

In [9]:
df = drop_period_after_war(df)

In [10]:
def merge_oil_to_data(data, oil):
    data['date_without_time'] = pd.to_datetime(data['Transaction Date']).dt.strftime('%Y-%m-%d')
    oil['date_without_time'] = pd.to_datetime(oil['date']).dt.strftime('%Y-%m-%d')
    data = data.merge(oil, left_on='date_without_time', right_on='date_without_time')
    return data

In [11]:
df = merge_oil_to_data(df, oil_data)

In [12]:
def find_length_from_cost(dubai_area_coordinates, point, data_area_name):
    cost = {'y': 25.732457, 'x': 55.090145}
    
    point_locations = dubai_area_coordinates[dubai_area_coordinates['geometry'].contains(point)]
    
    if point_locations.shape[0]:
        return {
            'data_area_name': data_area_name,
            'length_from_coast': ((point.x-cost['x'])**2+(point.y-cost['y'])**2)**(1/2)
        }
        
    return None

def add_distance_from_coast_online(data, dubai_area_coordinates):
    areas = data['Area'].unique()
    merged_list = []

    for i in areas:
        point = gpd.tools.geocode(i).loc[0]['geometry']
        
        point_location_object = find_length_from_cost(dubai_area_coordinates, point, i)
        if point_location_object:
            merged_list = merged_list + [point_location_object]
        else:
            point = gpd.tools.geocode('Dubai ' + i).loc[0]['geometry']
            point_location_object = find_length_from_cost(dubai_area_coordinates, point, i)
            if point_location_object:
                merged_list = merged_list + [point_location_object]
    
    lenghts_from_cost = pd.DataFrame(merged_list)
    
    lenghts_from_cost.to_csv('length_from_coast.csv')
    
    data = data.merge(lenghts_from_cost, left_on='Area', right_on='data_area_name')
    
    return data
    

In [13]:
def add_distance_from_coast_from_file(data):
    lenghts_from_cost = pd.read_csv('length_from_coast.csv')
    data = data.merge(lenghts_from_cost, left_on='Area', right_on='data_area_name')
    
    return data

In [14]:
# add_distance_from_coast_online(df, dubai_area_coordinates)
df = add_distance_from_coast_from_file(df)

In [15]:
def change_variables_essence(data):
    data['Nearest Metro'] = data['Nearest Metro'].apply(lambda m: 1 if m else 0)
    data['Nearest Mall'] = data['Nearest Mall'].apply(lambda m: 1 if m else 0)  
    return data

In [16]:
df = change_variables_essence(df)

In [17]:
df.head(2)

,Transaction Date,Transaction Type,Usage,Area,Property Sub Type,Amount,Property Size (sq.m),Nearest Metro,Nearest Mall,Nearest Landmark,date_without_time,date,Oil Price,Unnamed: 0,data_area_name,length_from_coast
0,2021-03-02 13:53:10,Mortgage,Residential,AL BARARI,Flat,1435909.09,138.93,1,1,IMG World Adventures,2021-03-02,2021-03-02T00:00:00,59.75,0,AL BARARI,0.670569
1,2021-03-02 13:53:10,Mortgage,Residential,AL BARARI,Flat,1435909.09,87.26,1,1,IMG World Adventures,2021-03-02,2021-03-02T00:00:00,59.75,0,AL BARARI,0.670569


In [18]:
def clean_outliers_in_data(data):
    quantiles = data.quantile(0.98)
    data = data[(data['Amount'] < quantiles['Amount']) & (data['Property Size (sq.m)'] < quantiles['Property Size (sq.m)'])]
    data = data[data['Property Sub Type'].isin(["Commercial", "Flat", "Hotel Apartment", "Hotel Rooms",  "Office", "Residential", "Residential / Attached Villas","Residential Flats", "Stacked Townhouses", "Villa"])]
    data = data.dropna()
    return data

In [19]:
cleaned_df = clean_outliers_in_data(df)

C:\Users\idtyu\AppData\Local\Temp\ipykernel_30772\708482841.py:2: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  quantiles = data.quantile(0.98)


In [20]:
cleaned_df

,Transaction Date,Transaction Type,Usage,Area,Property Sub Type,Amount,Property Size (sq.m),Nearest Metro,Nearest Mall,Nearest Landmark,date_without_time,date,Oil Price,Unnamed: 0,data_area_name,length_from_coast
0,2021-03-02 13:53:10,Mortgage,Residential,AL BARARI,Flat,1435909.09,138.93,1,1,IMG World Adventures,2021-03-02,2021-03-02T00:00:00,59.75,0,AL BARARI,0.670569
1,2021-03-02 13:53:10,Mortgage,Residential,AL BARARI,Flat,1435909.09,87.26,1,1,IMG World Adventures,2021-03-02,2021-03-02T00:00:00,59.75,0,AL BARARI,0.670569
2,2021-03-02 13:53:10,Mortgage,Residential,AL BARARI,Flat,1435909.09,76.13,1,1,IMG World Adventures,2021-03-02,2021-03-02T00:00:00,59.75,0,AL BARARI,0.670569
3,2021-03-02 13:53:10,Mortgage,Residential,AL BARARI,Flat,1435909.09,130.05,1,1,IMG World Adventures,2021-03-02,2021-03-02T00:00:00,59.75,0,AL BARARI,0.670569
4,2021-03-02 13:53:10,Mortgage,Residential,AL BARARI,Flat,1435909.09,52.15,1,1,IMG World Adventures,2021-03-02,2021-03-02T00:00:00,59.75,0,AL BARARI,0.670569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67082,2021-05-31 14:59:25,Sales,Commercial,Al Qusais Industrial Fourth,Commercial,9500000.00,1302.87,1,1,Dubai International Airport,2021-05-31,2021-05-31T00:00:00,66.93,193,Al Qusais Industrial Fourth,0.537750
67108,2021-04-07 13:02:45,Sales,Residential,Al Saffa First,Residential,6800000.00,1525.93,1,1,Downtown Dubai,2021-04-07,2021-04-07T00:00:00,59.77,204,Al Saffa First,0.660583
67109,2021-04-26 11:55:45,Sales,Residential,Al Saffa First,Residential,2800000.00,1393.55,1,1,Downtown Dubai,2021-04-26,2021-04-26T00:00:00,61.91,204,Al Saffa First,0.660583
67110,2021-05-17 10:10:56,Sales,Residential,Al Saffa First,Residential,5000000.00,1393.55,1,1,Downtown Dubai,2021-05-17,2021-05-17T00:00:00,66.27,204,Al Saffa First,0.660583


In [21]:
from pandas.plotting import scatter_matrix

def plot_matrix(data):
    scatter_matrix(data[['Amount', 'Property Size (sq.m)']], figsize=(12, 8))
    return data['Property Size (sq.m)'].describe()

In [ ]:
plot_matrix(cleaned_df)

In [22]:
def preparing_data_before_training(data, drop, renameColumns):
    data = data.drop(drop, axis=1)
    data = data.rename(columns=renameColumns)
    return data

In [23]:
prepared_data = preparing_data_before_training(
    cleaned_df, 
    ['date_without_time', 'Transaction Date', 'date'],
    {
        'Property Size (sq.m)': 'Property_Size',
        'Property Sub Type': 'Property_Sub_Type',
        'Nearest Metro': 'Nearest_Metro',
        'Nearest Mall': 'Nearest_Mall',
        'Nearest Landmark': 'Nearest_Landmark',
        'Oil Price': 'Oil_Price',
        'Transaction Type': 'Transaction_Type',
        'length_from_coast': 'Length_From_Coast'
    }
)

In [24]:
import statsmodels.formula.api as smf

sm_data = sm.add_constant(prepared_data)
first_model = smf.ols(formula='Amount ~ C(Transaction_Type) + C(Usage) + C(Area) + C(Property_Sub_Type) + Property_Size + C(Nearest_Metro) + C(Nearest_Mall) + C(Nearest_Landmark)', data=sm_data)
first_results = first_model.fit()


In [25]:
first_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Amount   R-squared:                       0.643
Model:                            OLS   Adj. R-squared:                  0.642
Method:                 Least Squares   F-statistic:                     558.0
Date:                Mon, 13 Mar 2023   Prob (F-statistic):               0.00
Time:                        04:44:09   Log-Likelihood:            -8.2867e+05
No. Observations:               54024   AIC:                         1.658e+06
Df Residuals:                   53849   BIC:                         1.659e+06
Df Model:                         174                                         
Covariance Type:            nonrobust                                         
=======================================================================================================================
                                                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
Intercept                                           -3.847e+05   1.26e+05     -3.049      0.002   -6.32e+05   -1.37e+05
C(Transaction_Type)[T.Mortgage]                     -1.091e+05   2.55e+04     -4.276      0.000   -1.59e+05   -5.91e+04
C(Transaction_Type)[T.Sales]                         5.431e+05   2.46e+04     22.112      0.000    4.95e+05    5.91e+05
C(Usage)[T.Residential]                             -2.399e+05   4.06e+04     -5.913      0.000   -3.19e+05    -1.6e+05
C(Area)[T.AL BARARI]                                 6.707e+05   1.46e+05      4.588      0.000    3.84e+05    9.57e+05
C(Area)[T.AL FURJAN]                                 8.674e+04   1.51e+05      0.576      0.565   -2.09e+05    3.82e+05
C(Area)[T.AL KHAIL HEIGHTS]                          2.418e+06   2.26e+05     10.696      0.000    1.97e+06    2.86e+06
C(Area)[T.AL WAHA]                                  -4.203e+05   1.39e+05     -3.017      0.003   -6.93e+05   -1.47e+05
C(Area)[T.ARABIAN RANCHES I]                        -2.508e+05   1.28e+05     -1.966      0.049   -5.01e+05    -780.998
C(Area)[T.ARABIAN RANCHES II]                        1.049e+05   1.32e+05      0.792      0.429   -1.55e+05    3.65e+05
C(Area)[T.ARABIAN RANCHES III]                       1.057e+05   1.71e+05      0.618      0.537    -2.3e+05    4.41e+05
C(Area)[T.ARABIAN RANCHES POLO CLUB]                -1.538e+06   5.14e+05     -2.994      0.003   -2.55e+06   -5.31e+05
C(Area)[T.ARJAN]                                     3.233e+05   1.24e+05      2.599      0.009    7.95e+04    5.67e+05
C(Area)[T.Abu Hail]                                  2.387e+06   6.07e+05      3.929      0.000     1.2e+06    3.58e+06
C(Area)[T.Al Bada]                                    3.08e+06   3.66e+05      8.414      0.000    2.36e+06     3.8e+06
C(Area)[T.Al Baraha]                                 5.359e+06   6.35e+05      8.437      0.000    4.11e+06     6.6e+06
C(Area)[T.Al Barsha First]                          -5.354e+05   4.16e+05     -1.287      0.198   -1.35e+06     2.8e+05
C(Area)[T.Al Barsha Second]                         -4.469e+06   2.66e+05    -16.787      0.000   -4.99e+06   -3.95e+06
C(Area)[T.Al Barsha Third]                           -4.05e+06   2.63e+05    -15.381      0.000   -4.57e+06   -3.53e+06
C(Area)[T.Al Barshaa South First]                   -4.172e+06   4.38e+05     -9.518      0.000   -5.03e+06   -3.31e+06
C(Area)[T.Al Barshaa South Third]                   -9.687e+04   6.53e+05     -0.148      0.882   -1.38e+06    1.18e+06
C(Area)[T.Al Dhagaya]                                5.143e+06   7.82e+05      6.578      0.000    3.61e+06    6.68e+06
C(Area)[T.Al Garhoud]                                3.276e+06   6.74e+05      4.86

In [26]:
sm_data = sm.add_constant(prepared_data)
second_model = smf.ols(formula='Amount ~ C(Transaction_Type) + C(Usage) + C(Area) + C(Property_Sub_Type) + Property_Size + C(Nearest_Metro) + C(Nearest_Mall) + C(Nearest_Landmark) + Oil_Price + Length_From_Coast', data=sm_data)
second_results = second_model.fit()

# formula = 'Amount ~ C(Transaction_Type) + C(Usage) + C(Area) + C(Property_Sub_Type) + Property_Size + C(Nearest_Metro) + C(Nearest_Mall) + C(Nearest_Landmark) + Oil_Price + Length_From_Coast'

In [ ]:
second_results.summary()

In [ ]:
pred_ols = second_results.get_prediction()

fig, ax = plt.subplots(figsize=(8, 6))

x = prepared_data['Property_Size']
y = prepared_data['Amount']

ax.plot(x, y, "o", label="data")
ax.plot(x, second_results.fittedvalues, "o", label="OLS")

ax.legend(loc="best")

In [50]:
prepared_data['Property_Size_Squared'] = prepared_data['Property_Size']**2

In [51]:
sm_data = sm.add_constant(prepared_data)
third_model = smf.ols(formula='Amount ~ C(Transaction_Type) + C(Usage) + C(Area) + C(Property_Sub_Type) + Property_Size + Property_Size_Squared + C(Nearest_Metro) + C(Nearest_Mall) + C(Nearest_Landmark) + Oil_Price + Length_From_Coast', data=sm_data)
third_results = third_model.fit()

# 'Amount ~ C(Transaction_Type) + C(Usage) + C(Area) + C(Property_Sub_Type) + Property_Size + Property_Size_Squared + C(Nearest_Metro) + C(Nearest_Mall) + C(Nearest_Landmark) + Oil_Price + Length_From_Coast'

In [52]:
third_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Amount   R-squared:                       0.634
Model:                            OLS   Adj. R-squared:                  0.633
Method:                 Least Squares   F-statistic:                     533.9
Date:                Mon, 13 Mar 2023   Prob (F-statistic):               0.00
Time:                        04:52:42   Log-Likelihood:            -8.2933e+05
No. Observations:               54024   AIC:                         1.659e+06
Df Residuals:                   53848   BIC:                         1.661e+06
Df Model:                         175                                         
Covariance Type:            nonrobust                                         
=======================================================================================================================
                                                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
Intercept                                           -5.991e+05   6.19e+05     -0.969      0.333   -1.81e+06    6.13e+05
C(Transaction_Type)[T.Mortgage]                     -2.641e+05   2.58e+04    -10.223      0.000   -3.15e+05   -2.14e+05
C(Transaction_Type)[T.Sales]                         5.463e+05   2.49e+04     21.966      0.000    4.98e+05    5.95e+05
C(Usage)[T.Residential]                             -7.254e+05   4.02e+04    -18.051      0.000   -8.04e+05   -6.47e+05
C(Area)[T.AL BARARI]                                -3.838e+04   1.46e+05     -0.264      0.792   -3.24e+05    2.47e+05
C(Area)[T.AL FURJAN]                                -3.817e+05   1.42e+05     -2.691      0.007    -6.6e+05   -1.04e+05
C(Area)[T.AL KHAIL HEIGHTS]                          1.389e+05   1.92e+05      0.724      0.469   -2.37e+05    5.15e+05
C(Area)[T.AL WAHA]                                  -7.714e+05   1.57e+05     -4.902      0.000   -1.08e+06   -4.63e+05
C(Area)[T.ARABIAN RANCHES I]                         8.323e+04   1.17e+05      0.709      0.479   -1.47e+05    3.13e+05
C(Area)[T.ARABIAN RANCHES II]                       -1.197e+05   1.24e+05     -0.964      0.335   -3.63e+05    1.24e+05
C(Area)[T.ARABIAN RANCHES III]                      -2.948e+05   1.69e+05     -1.745      0.081   -6.26e+05    3.63e+04
C(Area)[T.ARABIAN RANCHES POLO CLUB]                 2.763e+06   5.16e+05      5.357      0.000    1.75e+06    3.77e+06
C(Area)[T.ARJAN]                                     1.523e+05   1.25e+05      1.216      0.224   -9.32e+04    3.98e+05
C(Area)[T.Abu Hail]                                 -1.618e+06   4.09e+05     -3.959      0.000   -2.42e+06   -8.17e+05
C(Area)[T.Al Bada]                                  -1.109e+05   3.58e+05     -0.310      0.757   -8.12e+05    5.91e+05
C(Area)[T.Al Baraha]                                 1.467e+06   4.49e+05      3.267      0.001    5.87e+05    2.35e+06
C(Area)[T.Al Barsha First]                            4.83e+05   4.25e+05      1.136      0.256   -3.51e+05    1.32e+06
C(Area)[T.Al Barsha Second]                         -1.475e+06   2.72e+05     -5.422      0.000   -2.01e+06   -9.42e+05
C(Area)[T.Al Barsha Third]                           -1.42e+06   2.76e+05     -5.138      0.000   -1.96e+06   -8.78e+05
C(Area)[T.Al Barshaa South First]                    -2.04e+06    4.5e+05     -4.535      0.000   -2.92e+06   -1.16e+06
C(Area)[T.Al Barshaa South Third]                   -6.958e+05    6.6e+05     -1.055      0.292   -1.99e+06    5.97e+05
C(Area)[T.Al Dhagaya]                                7.031e+05   6.39e+05      1.100      0.271   -5.49e+05    1.96e+06
C(Area)[T.Al Garhoud]                                5.088e+05   5.23e+05      0.97

## TESTS

In [53]:
ramsey_test_1 = linear_reset(first_results)
ramsey_test_2 = linear_reset(second_results)
ramsey_test_3 = linear_reset(third_results)

C:\Users\idtyu\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\stats\diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


In [54]:
# H0 = Model correctly spicified
# H1 = Model misspecified
print(ramsey_test_1)
print(ramsey_test_2)
print(ramsey_test_3)

<Wald test (chi2): statistic=169077.13836277346, p-value=0.0, df_denom=2>
<Wald test (chi2): statistic=170220.82952358932, p-value=0.0, df_denom=2>
<Wald test (chi2): statistic=217058.5768198107, p-value=0.0, df_denom=2>


In [125]:
prepared_data_four = prepared_data
prepared_data_four['Price_per_sqm'] = prepared_data_four['Amount'] / prepared_data_four['Property_Size']
sm_data_four = sm.add_constant(prepared_data)

# 'Price_per_sqm ~ C(Transaction_Type) + C(Usage) + C(Area) + C(Property_Sub_Type) + C(Nearest_Metro) + C(Nearest_Mall) + C(Nearest_Landmark) + Oil_Price + Length_From_Coast'

four_model_formula = 'Price_per_sqm ~ C(Usage) + C(Area) + C(Property_Sub_Type) + Oil_Price + Length_From_Coast'

four_model = smf.ols(formula=four_model_formula, data=sm_data_four)

four_results = four_model.fit()

In [126]:
four_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Price_per_sqm   R-squared:                       0.552
Model:                            OLS   Adj. R-squared:                  0.550
Method:                 Least Squares   F-statistic:                     411.8
Date:                Mon, 13 Mar 2023   Prob (F-statistic):               0.00
Time:                        05:45:58   Log-Likelihood:            -5.4288e+05
No. Observations:               54024   AIC:                         1.086e+06
Df Residuals:                   53862   BIC:                         1.088e+06
Df Model:                         161                                         
Covariance Type:            nonrobust                                         
==============================================================================================================
                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
Intercept                                    1.01e+04    670.515     15.064      0.000    8786.693    1.14e+04
C(Usage)[T.Residential]                     2651.4275    191.151     13.871      0.000    2276.770    3026.085
C(Area)[T.AL BARARI]                        1037.1007    302.193      3.432      0.001     444.800    1629.401
C(Area)[T.AL FURJAN]                       -1964.3359    216.912     -9.056      0.000   -2389.486   -1539.186
C(Area)[T.AL KHAIL HEIGHTS]                -7244.3041    439.618    -16.479      0.000   -8105.959   -6382.649
C(Area)[T.AL WAHA]                         -5032.4658    685.380     -7.343      0.000   -6375.815   -3689.116
C(Area)[T.ARABIAN RANCHES I]               -2095.1260    243.025     -8.621      0.000   -2571.456   -1618.795
C(Area)[T.ARABIAN RANCHES II]               -406.4653    320.508     -1.268      0.205   -1034.663     221.732
C(Area)[T.ARABIAN RANCHES III]               944.9928    310.591      3.043      0.002     336.233    1553.753
C(Area)[T.ARABIAN RANCHES POLO CLUB]       -2330.4976   2505.857     -0.930      0.352   -7241.997    2581.002
C(Area)[T.ARJAN]                            -895.0714    230.813     -3.878      0.000   -1347.467    -442.675
C(Area)[T.Abu Hail]                        -4152.5243   1321.717     -3.142      0.002   -6743.101   -1561.948
C(Area)[T.Al Bada]                          4515.7136   1496.095      3.018      0.003    1583.355    7448.072
C(Area)[T.Al Baraha]                        8986.4032   1611.914      5.575      0.000    5827.039    1.21e+04
C(Area)[T.Al Barsha First]                 -1276.8992   1978.263     -0.645      0.519   -5154.311    2600.513
C(Area)[T.Al Barsha Second]                -5030.6104   1148.127     -4.382      0.000   -7280.949   -2780.272
C(Area)[T.Al Barsha Third]                 -5220.0983   1125.333     -4.639      0.000   -7425.759   -3014.437
C(Area)[T.Al Barshaa South First]          -6012.6269   2114.464     -2.844      0.004   -1.02e+04   -1868.260
C(Area)[T.Al Barshaa South Third]          -6148.2130   3233.360     -1.901      0.057   -1.25e+04     189.199
C(Area)[T.Al Dhagaya]                       1.493e+04   2779.246      5.372      0.000    9481.741    2.04e+04
C(Area)[T.Al Garhoud]                      -3438.8826   1971.877     -1.744      0.081   -7303.778     426.013
C(Area)[T.Al Hamriya]                       3802.0407   2108.478      1.803      0.071    -330.593    7934.675
C(Area)[T.Al Hudaiba]                      -7588.8941   3926.334     -1.933      0.053   -1.53e+04     106.752
C(Area)[T.Al Jafliya]                      -3234.6484   1860.522     -1.739      0.082   -6881.286     411.989
C(Area)[T.Al Karama]                        7490.7816   1618.785      4.627      0.000

In [127]:
ramsey_test_4 = linear_reset(four_results)

C:\Users\idtyu\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\stats\diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


In [128]:
ramsey_test_4

<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=2463.4485751214897, p-value=0.0, df_denom=2>

In [129]:
four_results_robust_HC0 = four_model.fit(cov_type='HC0')

In [130]:
four_results_robust_HC0.summary()

C:\Users\idtyu\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 163, but rank is 158
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Price_per_sqm   R-squared:                       0.552
Model:                            OLS   Adj. R-squared:                  0.550
Method:                 Least Squares   F-statistic:                     4273.
Date:                Mon, 13 Mar 2023   Prob (F-statistic):               0.00
Time:                        05:46:11   Log-Likelihood:            -5.4288e+05
No. Observations:               54024   AIC:                         1.086e+06
Df Residuals:                   53862   BIC:                         1.088e+06
Df Model:                         161                                         
Covariance Type:                  HC0                                         
==============================================================================================================
                                                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
Intercept                                    1.01e+04    491.517     20.550      0.000    9137.552    1.11e+04
C(Usage)[T.Residential]                     2651.4275    115.197     23.016      0.000    2425.646    2877.209
C(Area)[T.AL BARARI]                        1037.1007    221.084      4.691      0.000     603.785    1470.416
C(Area)[T.AL FURJAN]                       -1964.3359    115.309    -17.035      0.000   -2190.337   -1738.335
C(Area)[T.AL KHAIL HEIGHTS]                -7244.3041    164.001    -44.172      0.000   -7565.740   -6922.868
C(Area)[T.AL WAHA]                         -5032.4658    158.897    -31.671      0.000   -5343.898   -4721.034
C(Area)[T.ARABIAN RANCHES I]               -2095.1260     93.726    -22.354      0.000   -2278.826   -1911.426
C(Area)[T.ARABIAN RANCHES II]               -406.4653    138.409     -2.937      0.003    -677.742    -135.188
C(Area)[T.ARABIAN RANCHES III]               944.9928    125.079      7.555      0.000     699.842    1190.144
C(Area)[T.ARABIAN RANCHES POLO CLUB]       -2330.4976    781.269     -2.983      0.003   -3861.757    -799.238
C(Area)[T.ARJAN]                            -895.0714    135.533     -6.604      0.000   -1160.711    -629.432
C(Area)[T.Abu Hail]                        -4152.5243   1198.184     -3.466      0.001   -6500.922   -1804.126
C(Area)[T.Al Bada]                          4515.7136   4321.803      1.045      0.296   -3954.865     1.3e+04
C(Area)[T.Al Baraha]                        8986.4032   2848.031      3.155      0.002    3404.365    1.46e+04
C(Area)[T.Al Barsha First]                 -1276.8992   1799.220     -0.710      0.478   -4803.306    2249.508
C(Area)[T.Al Barsha Second]                -5030.6104    383.022    -13.134      0.000   -5781.320   -4279.900
C(Area)[T.Al Barsha Third]                 -5220.0983    483.801    -10.790      0.000   -6168.331   -4271.866
C(Area)[T.Al Barshaa South First]          -6012.6269    648.155     -9.277      0.000   -7282.988   -4742.266
C(Area)[T.Al Barshaa South Third]          -6148.2130    591.762    -10.390      0.000   -7308.046   -4988.380
C(Area)[T.Al Dhagaya]                       1.493e+04   1864.362      8.008      0.000    1.13e+04    1.86e+04
C(Area)[T.Al Garhoud]                      -3438.8826   1408.061     -2.442      0.015   -6198.631    -679.134
C(Area)[T.Al Hamriya]                       3802.0407   2745.073      1.385      0.166   -1578.204    9182.285
C(Area)[T.Al Hudaiba]                      -7588.8941    160.937    -47.154      0.000   -7904.325   -7273.463
C(Area)[T.Al Jafliya]                      -3234.6484   1949.682     -1.659      0.097   -7055.955     586.658
C(Area)[T.Al Karama]                        7490.7816   3487.341      2.148      0.032

In [131]:
ramsey_test_4_robust_HC0 = linear_reset(four_results_robust_HC0)

C:\Users\idtyu\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\stats\diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


In [132]:
ramsey_test_4_robust_HC0

<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=2463.4485751214897, p-value=0.0, df_denom=2>